# Hypothesis testing 2

We will be working with data gathered for Korean citizens. This dataset is concerned with respondent's household income and various socio-economic features:

There are 14 columns in data;

* **id**
* **year** study was conducted
* **wave** from wave 1st in 2005 to wave 14th in 2018
* **region** 1) Seoul 2) Kyeong-gi 3) Kyoung-nam 4) Kyoung-buk 5) Chung-nam 6) Gang-won & Chung-buk 7) Jeolla & Jeju
* **income** yearly income in M KRW(Million Korean Won. 1100 KRW = 1 USD)
* **family_member** number of family members
* **gender** 1) male 2) female
* **year_born**
* **education_level** 1) no education(under 7 yrs-old) 2) no education(7 & over 7 yrs-old) 3) elementary 4) middle school 5) high school 6) college 7) university degree 8) MA 9) doctoral degree
* **marriage** marital status. 1) not applicable (under 18) 2) married 3) separated by death 4) separated 5) not married yet 6) others
* **religion** 1) have religion 2) do not have
* **occupation** this is provided in separate code book (we will not use this one)
* **company_size**
* **reason_none_worker** 1) not capable 2) in military service 3) studying in school 4) preparing for school 5) preparing to apply for a job 6) house worker 7) caring for kids at home 8) nursing 9) giving-up economic activities 10) no intention to work 11) others

In [1]:
import pandas as pd
from scipy import stats

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/rogovich/Data/master/data/Korea%20Income%20and%20Welfare.csv')
df.head()

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92857 entries, 0 to 92856
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  92857 non-null  int64  
 1   year                92857 non-null  int64  
 2   wave                92857 non-null  int64  
 3   region              92857 non-null  int64  
 4   income              92857 non-null  float64
 5   family_member       92857 non-null  int64  
 6   gender              92857 non-null  int64  
 7   year_born           92857 non-null  int64  
 8   education_level     92857 non-null  int64  
 9   marriage            92857 non-null  int64  
 10  religion            92857 non-null  int64  
 11  occupation          92857 non-null  object 
 12  company_size        92857 non-null  object 
 13  reason_none_worker  92857 non-null  object 
dtypes: float64(1), int64(10), object(3)
memory usage: 9.9+ MB


## One-sample t-test

Suppose we are conducting study for our sales department. We want to enter national market and therefore we are interested in Koreans' average salary. Let's compute it:

In [4]:
income_average = df['income'].mean()
income_average

3441.1223268686776

We've computed it, but we want to add pretty round value in our report, not this one. For example, we round 3441 to 3500. Is it justifiable from statistical perspective? It's time to check. We have "real" mean already. Now remind the formula:

$\frac{M - \mu}{SEM}\$,

where $\mu$ is theoretical population mean (3500 in our case), $M$ is "real" mean, $SEM$ is standard mean error ($\frac{std}{\sqrt{n}}\$).

In [5]:
theoretic_mean = 3500

difference = income_average - theoretic_mean
difference

-58.877673131322354

So, we've computed difference between 'real' and 'theoretic' means: nearly 59. Now we have two alternatives:

1. This difference between 'real' (sample) and 'theoretic' (general) means emerged by chance;
2. This difference between 'real' (sample) and 'theoretic' (general) means is too extreme to occur accidentally.

Let's take first variant as a starting point. What is a probability to get such a difference by chance? To answer this question, we'd probably like to know what is regarded as an "extreme" difference. This [table](https://www.tdistributiontable.com/) will show us the answer.

In [6]:
SEM = df['income'].std() / (df['income']. size ** (1 / 2))

statistic = difference / SEM

print(statistic)

-4.295994574110945


Now we should interpret this statistic. Let's use the [table](https://www.tdistributiontable.com/).

By the way, to answer this question we could use function from `scipy.stats` module called `ttest_1samp`:

In [7]:
stats.ttest_1samp(
    df['income'].values,  # sample from which we derive sample mean
    theoretic_mean,       # theoretic mean
    alternative='less'    # alternative hypothesis. Initial hypothesis is that theoretic (3500) is HIGHER than sample (3441), so the
                          # the alternative is LESS
)
# statistic value should be taken as an absolute because normal distibution is symmetric

TtestResult(statistic=-4.295994574110945, pvalue=8.70441723081711e-06, df=92856)

## Independent (Two) samples t-test

Now let's consider two samples: males' and females' income in 2018:

In [8]:
df.head()

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10


In [9]:
male_income2018 = df[(df['gender'] == 1) & (df['year'] == 2018)]['income'].values
female_income2018 = df[(df['gender'] == 2) & (df['year'] == 2018)]['income'].values

print(male_income2018.mean().round(), female_income2018.mean().round())

5414.0 2089.0


We are going to test the hypothesis that these means are not equal. In the case our null and alternative hypotheses will be as follows:

$ H_0: \mu_{male} = \mu_{female} $

$ H_a: \mu_{male} \neq \mu_{female} $

First, we should select critical values of statistic that will be regarded as "extreme". From theory we know that statistics has t-distribution with degrees of freedom (df) equals to (`n + m - 2`), where `n` and `m` are sample sizes. We again consult [t-distribution table](https://www.tdistributiontable.com/) to find appropriate values:

In [10]:
n = male_income2018.size
m = female_income2018.size
degrees_of_freedom = n + m - 2

In `df` column we should choose `Z` value as our `degrees_of_freedom` is higher than 1000. To select critical "extreme" values, let us initially set the alpha level to traditional 5% (0.05). Since our alternative hypothesis is two-sided ($\mu_{male} \neq \mu_{female}$, not <b>>=</b> or <b><=</b>), we can have both extremely low and extremely high values. Therefore, our critical points should have area of 0.025 from the left and from the right (0.05 / 2 -> 0.025) accordingly:

critical_left: -1.96<br>
critical_right: 1.96

Again, we could use `stats` module instead of table to find critical values:

In [11]:
statistic_distrib = stats.norm(  # t-distribution -> standard normal distribution with large n
    0,  # mean
    1   # std
)

critical_points = statistic_distrib.ppf(  # computes points based on area ONLY from the left
    [
        0.025,    # left area to compute LEFT critical point
        1 - 0.025 # left area to compute RIGHT critical point
    ]
)

critical_points

array([-1.95996398,  1.95996398])

After we've defined critical points, time to compute real statistic value according to the formula:

In [12]:
difference = male_income2018.mean() - female_income2018.mean()
SEM = (male_income2018.var(ddof=1) / male_income2018.size + female_income2018.var(ddof=1) / female_income2018.size) ** (1 / 2)

statistic = difference / SEM

print(statistic)  # really extreme...

37.65280731063825


In [13]:
# use stats again to test the same
stats.ttest_ind(
    male_income2018,         # first sample
    female_income2018,       # second sample
    alternative='two-sided', # alternative hypothesis type
    equal_var=False          # special option
)

TtestResult(statistic=37.65280731063825, pvalue=1.6932116672305666e-278, df=5952.8172540662)

## Confidence intervals

In [14]:
mean_fi18 = female_income2018.mean()
alpha = 0.05
z = statistic_distrib.ppf(1 - alpha / 2)
SEM = mean_fi18 / female_income2018.std() ** (1 / 2)

left_border = mean_fi18 - z * SEM
right_border = mean_fi18 + z * SEM

print(left_border, mean_fi18, right_border)

1996.251856561804 2089.102809765085 2181.9537629683664


## Chi-Square test for Independence

Let's investigate a slightly more complicated question: **is there some relationship between gender and education level?** Both of those variables have categories. Let's filter only the data we need and also let's take only the year 2018.

We also convert our frequency table to the contigency table format via `.unstack()` method.

In [15]:
data_edu_gender = df[df['year'] == 2018].groupby('education_level')['gender'].value_counts().unstack()

So, we have the following observed frequencies:

In [16]:
data_edu_gender

gender,1,2
education_level,,
2,114,528
3,640,797
4,571,292
5,1396,329
6,392,81
7,873,123
8,152,15
9,22,6


Here is a reminder how `education_level` is coded:

* **education_level** 1) no education(under 7 yrs-old) 2) no education(7 & over 7 yrs-old) 3) elementary 4) middle school 5) high school 6) college 7) university degree 8) MA 9) doctoral degree

Let's formulate hypotheses:

**Null hypothesis ($H_0$)**: For the general population in 2018 of Koreans there was no relationship between gender and education level.

**Alternative hypothesis ($H_1$)**: For the general population of Koreans in 2018 there was a relationship between gender and education level.

Now we need to choose an **alpha level** and calculate the degrees of freedom. Let's do the rigorous test and choose **$\alpha = 0.05$**. This means that there is a 5% chance to make a mistake in our conclusion about population due to the 'unfortunate' sampling.

Next step is to calculate **degrees of freedom**.

For Chi-square test of Independence we do it like this:
$df = (C - 1)(R - 1)$ where $C$ is the number of columns, and $R$ is number of rows.

In [17]:
ddof = (data_edu_gender.shape[1] - 1) * (data_edu_gender.shape[0] - 1)
ddof

7

Now we need to look up Chi-square value that would be enough for us to reject a null hypothesis. Let's find it in the [table](https://people.richland.edu/james/lecture/m170/tbl-chi.html).

The value separating critical region is $\chi^2 = 14.067$.

Let's update our function for hypothesis check:

In [18]:
def check_hypothesis(chi_square):
    if chi_square < 14.067:
        print('Null hypothesis is not rejected')
    else:
        print('Null hypothesis is rejected')

The next step would be to find expected frequencies for each cell of our contigency table. Here it would be a tedious process but let's do one row for example. Let's consider category 9 — doctoral degrees.

First, we need to find four numbers:
* total number of respondents
* 1st column total (all men)
* 2nd column total (all women)
* row total (all respondents with doctoral degree)


In [19]:
total = data_edu_gender.sum().sum()
men = data_edu_gender[1].sum()
women = data_edu_gender[2].sum()
doctoral = data_edu_gender.loc[9].sum()
print(total, men, women, doctoral)

6331 4160 2171 28


Now to calculate expected frequencies for men and women with doctoral degrees, we have to find what is the proportion of respondents have a doctoral degree:

In [20]:
p = doctoral / total
print(p)

0.004422682040751856


Then in accordance with the null hypothesis of no relationship we would expect that the same proportions of men and women do have a doctoral degree

In [21]:
print('Men, doctoral degree, expected frequency:')
print(men * p)

print('Women, doctoral degree, expected frequency:')
print(women * p)

Men, doctoral degree, expected frequency:
18.39835728952772
Women, doctoral degree, expected frequency:
9.601642710472278


So our observed frequencies are 22 and 6 for men and women respectevely, and expected are ~18 and ~9.

Let's find all the expected frequencies with `chi2_contingency` function from `scipy.stats`. It takes as an argument contigency table of expected frequencies and returns a chi-square statistic, p-value, degrees of freedom and a contigency table of expected frequencies.

In [22]:
stats.chi2_contingency(data_edu_gender)

Chi2ContingencyResult(statistic=1442.5981765341737, pvalue=2.338859768704989e-307, dof=7, expected_freq=array([[ 421.84804928,  220.15195072],
       [ 944.22997947,  492.77002053],
       [ 567.06365503,  295.93634497],
       [1133.47022587,  591.52977413],
       [ 310.80082136,  162.19917864],
       [ 654.45585216,  341.54414784],
       [ 109.73305955,   57.26694045],
       [  18.39835729,    9.60164271]]))

Check the last row! Did we calculate it correctly?

We always can derive a chi-square from the function's output and make a conclusion.

In [23]:
chi_square = stats.chi2_contingency(data_edu_gender)[0]
print('Chi-Square:', chi_square)
check_hypothesis(chi_square)

Chi-Square: 1442.5981765341737
Null hypothesis is rejected


## Practice

**NB!** Algoritm for successful practice solution is as follows:

1. Define types of the variables you've encountered (categorical, numeric);
2. Formulate the Null and Alternative Hypotheses ($H_0$ and $H_a$);
3. Choose statistic;
4. Choose critical value of statistic;
5. Compute observed value of statistic;
6. Make both statistical and substantial conclusions.

**Task 1.** Is there any relation between relegiousness and gender? Do not use `scipy.stats` module.

In [25]:
df['religion']. corr(df['gender'])# your code here

-0.13817245945721415

There is no relationship between gender and religion

**Task 2**. Do you remember case with t-test from the beginning of seminar? Find out whether it is correct to round 3441 to 3450. Do not use `scipy.stats` module.

In [26]:
income_average = df['income'].mean()# your code here
theoretic_mean = 3450
difference = income_average - theoretic_mean
SEM = df['income'].std() / (df['income']. size ** (1 / 2))
statistic = difference / SEM
statistic

-0.6477571815351186

Based on the t-test table, we will find the p-value. It is more than 30%. This means that we reject the hypothesis

**Task 3**. Remind previous task. Сonstruct a 99% confidence interval (alpha value equal to 0.01) of the income mean 3441. You should use `norm.interval` method from `stats` module for this task.

In [38]:
statistic_distrib = stats.norm.interval(0.01, 3441, SEM)
statistic_distrib

(3440.8282256860043, 3441.1717743139957)

**Task 4**. Is education level distribution different across regions? You should use `scipy.stats` module for this task.

In [44]:
data_edu_region = df[df['year'] == 2018].groupby('education_level')['region'].value_counts().unstack()
ddof = (data_edu_region.shape[1] - 1) * (data_edu_region.shape[0] - 1) 
display(ddof) #x**2 == 55.758
def check_hypothesis_1(chi_square):
    if chi_square < 55.758:
        print('Null hypothesis is not rejected')
    else:
        print('Null hypothesis is rejected')
stats.chi2_contingency(data_edu_region)
chi_square = stats.chi2_contingency(data_edu_region)[0]
print('Chi-Square:', chi_square)
check_hypothesis_1(chi_square)

42

Chi-Square: 257.90376514653656
Null hypothesis is rejected


The result of the program tells us that the null hypothesis is rejected.

**Task 5**. Test the hypothesis that the income means of Korean citizens who have religion (coded as 1) and do not have religion (coded as 2) are equal using [ttest_ind](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) function.

In [57]:
a = df.query('religion == 1')
in_m1 = a['income']
b = df.query('religion != 2')
in_m2 = b['income']
stats.ttest_ind(in_m1, in_m2)

TtestResult(statistic=-0.00157459943034313, pvalue=0.9987436552943652, df=93753.77318242136)

p-value ~99.8%. the value is greater than alpha, which means we reject the null hypothesis